In [63]:
import pandas as pd
import numpy as np
#import datetime
from datetime import datetime

In [2]:
data_fuel_events = pd.read_csv('data/Extranet2.ExactFuelEvents.csv')

In [3]:
data_fuel_tank_levels = pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')

In [118]:
data_fuel_transaction = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')

In [ ]:
data_fuel_transaction.shape

In [27]:
data_units = pd.read_csv('data/IBGEFILE.UNITS.csv')

In [28]:
data_units.shape

(709, 3)

In [31]:
data_units.dtypes


UNUNIT    object
UNYEAR    object
UNMAKE    object
dtype: object

In [32]:
data_units.columns = ['unit_number', 'model_year', 'make']

In [33]:
data_units['make']= data_units['make'].astype('category')

In [34]:
#data_units.unit_number.unique()
data_units.unit_number[~data_units.applymap(np.isreal).all(1)]

0       535
1       534
2       533
3       532
4       531
5       526
6       238
7       536
8       537
9       538
10      539
11      540
12        1
13        1
14      121
15     1505
16     1506
17     1507
18     1508
19     1509
20      243
21      244
22      245
23      246
24     1510
25     1511
26     1512
27     1513
28     1514
29     1515
       ... 
679    2147
680    2148
681    2149
682    2150
683    2151
684    2152
685    2153
686    2154
687    2155
688    2156
689    2157
690    2158
691    2159
692    2160
693    2161
694    2162
695    2163
696    2164
697    2165
698    2166
699    2167
700    2168
701    2169
702    2170
703    2171
704    2172
705    2173
706    2174
707    2175
708    2176
Name: unit_number, Length: 709, dtype: object

In [35]:

data_units.unit_number = pd.to_numeric(data_units.unit_number,errors='coerce')

In [36]:
data_units.dtypes

unit_number     float64
model_year       object
make           category
dtype: object

In [37]:
data_units.head(n=5)

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT
3,532.0,1996,PETERBILT
4,531.0,1996,PETERBILT


In [16]:
#striping the white spaces before and after the string
data_units.make.cat.categories.str.strip()

Index(['', 'FREIGHTLINER', 'KENOWRTH', 'KENWORTH', 'KEWORTH', 'MACK',
       'PETERBILT', 'SYSTEM TEST', 'VOLVO'],
      dtype='object')

In [38]:
data_units.make.value_counts()

KENWORTH        524
VOLVO           139
FREIGHTLINER     15
PETERBILT        14
MACK             10
                  4
SYSTEM TEST       1
KEWORTH           1
KENOWRTH          1
Name: make, dtype: int64

In [39]:
data_units = data_units[data_units.unit_number.isin(['ZEBE', 'CCTRK', 'BGXL', '1']) == False]

In [40]:
data_units.shape

(709, 3)

In [41]:
data_units = data_units[data_units.unit_number != 1].dropna()

In [42]:
data_units.columns

Index(['unit_number', 'model_year', 'make'], dtype='object')

In [43]:
data_units.head(n=5)

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT
3,532.0,1996,PETERBILT
4,531.0,1996,PETERBILT


In [44]:
data_units['make'].cat.rename_categories(data_units['make'].cat.categories.str.strip(), inplace=True)

In [45]:
#data_units.make[data_units.make.isin(['KEWORTH','KENOWRTH'])] = 'KENWORTH'
data_units.make[data_units.make.isin(['KENOWRTH','KEWORTH'])] = 'KENWORTH'

In [46]:
data_units.make.value_counts()

KENWORTH        526
VOLVO           139
FREIGHTLINER     15
PETERBILT        14
MACK             10
SYSTEM TEST       0
KEWORTH           0
KENOWRTH          0
                  0
Name: make, dtype: int64

In [47]:
data_units.head(n=5)

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT
3,532.0,1996,PETERBILT
4,531.0,1996,PETERBILT


In [48]:
data_units.isnull().any()

unit_number    False
model_year     False
make           False
dtype: bool

In [49]:
data_fuel_optimization = pd.read_csv('data/BGEIDSC.EF2EFFP.csv')

In [50]:
data_fuel_optimization.shape

(836, 7)

In [51]:
#column names to lower for easier typing
data_fuel_optimization.columns = [s.lower() for s in data_fuel_optimization.columns]
data_fuel_optimization.columns

Index(['truck', 'active', 'sendfuel', 'sendroute', 'tankcap', 'avgmpg',
       'sattype'],
      dtype='object')

In [52]:
data_fuel_optimization.tankcap.value_counts()

200    510
230    180
250    132
235     10
120      4
Name: tankcap, dtype: int64

In [ ]:
data_fuel_optimization.columns

In [53]:
opt_columns = ['active', 'sendfuel','sendroute','avgmpg','sattype']
data_fuel_optimization.drop(opt_columns, inplace=True, axis=1)

In [54]:
data_fuel_optimization.head(n=5)

,truck,tankcap
0,1,200
1,243,200
2,244,200
3,245,200
4,246,200


In [ ]:
data_fuel_optimization = data_fuel_optimization.set_index('truck')

In [119]:
#Actual fuel transactions will have an item code of:
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
# and could be in any of the item slots 1-6

In [120]:
data_fuel_transaction.columns = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6',
                         ]

In [121]:
data_fuel_transaction.dtypes

tr_id                int64
tr_date              int64
tr_time              int64
tr_station_id        int64
tr_station_name     object
tr_station_cty      object
tr_station_st       object
tr_drvr_code        object
tr_unit_id          object
tr_item_cd1         object
tr_item_qty1       float64
tr_item_cd2         object
tr_item_qty2       float64
tr_item_cd3         object
tr_item_qty3       float64
tr_item_cd4         object
tr_item_qty4       float64
tr_item_cd5         object
tr_item_qty5       float64
tr_item_cd6         object
tr_item_qty6         int64
dtype: object

In [150]:
data_fuel_transaction.memory_usage(index=True).sum() # previous memory 24840728

24840728

In [122]:
data_fuel_transaction.memory_usage(index=True).sum() # after changing the datatypes - 11017378 ...

24840728

In [123]:
'''tr_dtypes= {'tr_id' : 'uint32',  
            'tr_date' : 'uint32', 
            'tr_time' : 'uint32', 
            'tr_station_name' : 'object', 
            'tr_station_cty' : 'object', 
            'tr_station_st' : 'object', 
            'tr_drvr_code' : 'category', 
            'tr_unit_id' : 'category', 
            'tr_item_cd1' : 'category', 
            'tr_item_qty1' : 'float32', 
            'tr_item_cd2' : 'category', 
            'tr_item_qty2' : 'float32', 
            'tr_item_cd3' : 'category', 
            'tr_item_qty3' : 'float32', 
            'tr_item_cd4' : 'category', 
            'tr_item_qty4' : 'float32', 
            'tr_item_cd5' : 'category', 
            'tr_item_qty5' : 'float32', 
            'tr_item_cd6' : 'category', 
            'tr_item_qty6' : 'float32'}
            '''

"tr_dtypes= {'tr_id' : 'uint32',  \n            'tr_date' : 'uint32', \n            'tr_time' : 'uint32', \n            'tr_station_name' : 'object', \n            'tr_station_cty' : 'object', \n            'tr_station_st' : 'object', \n            'tr_drvr_code' : 'category', \n            'tr_unit_id' : 'category', \n            'tr_item_cd1' : 'category', \n            'tr_item_qty1' : 'float32', \n            'tr_item_cd2' : 'category', \n            'tr_item_qty2' : 'float32', \n            'tr_item_cd3' : 'category', \n            'tr_item_qty3' : 'float32', \n            'tr_item_cd4' : 'category', \n            'tr_item_qty4' : 'float32', \n            'tr_item_cd5' : 'category', \n            'tr_item_qty5' : 'float32', \n            'tr_item_cd6' : 'category', \n            'tr_item_qty6' : 'float32'}\n            "

In [124]:
data_fuel_transaction.tr_station_id = data_fuel_transaction.tr_station_id.astype('uint32')

In [125]:
data_fuel_transaction.columns

Index(['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
       'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id',
       'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
       'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
       'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6'],
      dtype='object')

In [127]:
#confirm no junk data in this col. 
None in data_fuel_transaction.tr_time

#Combine tr_date and tr_time into a single date time column
    #1. zero pad tr_time on left for interpretation as military time
        #cast tr_time as string
data_fuel_transaction.tr_time = data_fuel_transaction.tr_time.astype(str)

#ensure none of my strings are over 4 digits in length
for s in data_fuel_transaction.tr_time:
    assert len(s) < 5

#check for blanks
'' in data_fuel_transaction.tr_time

#add zeros to the left until each cell is len 4
zero_padded_time = []
for s in data_fuel_transaction.tr_time:
    while len(s) < 4:
        s = '0' + s
    zero_padded_time.append(s)

#spot check
for s in zero_padded_time:
    assert len(s) == 4

data_fuel_transaction.tr_time = zero_padded_time

#append tr_date (as string) and tr_time together, and then convert them to dateime
#transaction_df.tr_datetime 
tst = [datetime.strptime(str(d) + 
                         t, '%Y%m%d%H%M') 
       for d, t in zip(data_fuel_transaction.tr_date,
                       data_fuel_transaction.tr_time)]
#love list comprehensions

data_fuel_transaction['tr_datetime'] = tst#assign

#drop superfluous columns
#data_fuel_transaction.drop(['tr_date','tr_time'], axis = 'columns', inplace=True)
data_fuel_transaction.head()

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,20170101,0020,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 00:20:00
1,42575331,20170101,0048,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,7.71,CADV,1.0,,0.0,,0.0,,0,2017-01-01 00:48:00
2,42578678,20170101,0134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,4.12,,0.0,,0.0,,0.0,,0,2017-01-01 01:34:00
3,42578722,20170101,0137,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 01:37:00
4,42579695,20170101,0302,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 03:02:00


In [133]:
data_fuel_transaction.tr_drvr_code.unique()

array(['ULAL  ', 'MANRI ', 'RUSR  ', 'FOSJA ', 'HICWIL', 'BATK  ',
       'MARJOH', 'JAML  ', 'BEAKEV', 'MCAD  ', 'JARJ  ', 'LESJ  ',
       'BLACH ', 'WILTED', 'HIGK  ', 'STEMI ', 'WICJO ', 'STED  ',
       'BARW  ', 'CHRN  ', 'STEGA ', 'GLAJ  ', 'LANA  ', 'FAIC  ',
       'UHLS  ', 'HARWI ', 'MARCY ', 'ECKS  ', 'TAYB  ', 'WRIJON',
       'COLMAR', 'ELLTH ', 'GOOR  ', 'BOUB  ', 'YOUW01', 'ANDR  ',
       'DAWDA ', 'FREDA ', 'COFS  ', 'COOLAW', 'WEBTH ', 'HARST ',
       'SWET  ', 'CLAMI ', 'ROBMA ', 'MORJAM', 'AUSTE ', 'EDEMA ',
       'CUNW  ', 'ROBJEF', 'CRAMI ', 'DIVH  ', 'SIMJE ', 'NORJI ',
       'LUNR  ', 'SMAD  ', 'LEWMA ', 'BOLJ  ', 'OAKR  ', 'FLIE  ',
       'BELT  ', 'THOCHA', 'BAIJO ', 'CROK  ', 'EVAG  ', 'TAYANT',
       'SCHC  ', 'LONRO ', 'BEAJI ', 'CARJEF', 'HATA  ', 'CARRO ',
       'SMIGA ', 'CARGA ', 'GERK  ', 'HOYJ  ', 'JOHTE ', 'RENT  ',
       'ANDJER', 'TAYJON', 'HOLS  ', 'PETL  ', 'MCMKE ', 'ADAR  ',
       'OWEJE ', 'SMIR01', 'DONR  ', 'FOSR  ', 'DYET  ', 'STIH

In [134]:
data_fuel_transaction[data_fuel_transaction['tr_drvr_code'] == 'ULAL  ']


,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,20170101,0020,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 00:20:00
110,42614106,20170101,1712,516130,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 17:12:00
282,42672413,20170102,1529,524100,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-02 15:29:00
851,42873010,20170104,1152,516012,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,...,1.00,,0.0,,0.0,,0.0,,0,2017-01-04 11:52:00
883,42882523,20170104,1300,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-04 13:00:00
2089,43280882,20170107,1520,231064,FJ-BEAVER DAM 695,BEAVER DAM,OH,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-07 15:20:00
2417,43394310,20170109,0203,231060,FJ-CARLISLE 708,CARLISLE,PA,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-09 02:03:00
3041,43596583,20170110,1509,231846,FJ-MAX MEADOWS 750,FORT CHISWELL,VA,ULAL,1564,ULSD,...,6.92,CADV,1.0,,0.0,,0.0,,0,2017-01-10 15:09:00
3424,43719948,20170111,1339,521442,TIMEOUT MARKET #22 & TC,NEWPORT,TN,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-11 13:39:00
4302,44020016,20170113,1645,516264,PILOT MURFREESBORO 404,MURFREESBORO,TN,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-13 16:45:00


In [66]:
data_fuel_transaction.tr_station_name.value_counts()

BIG G EXPRESS TERMINAL       12873
PILOT WHITE PINE 412          5978
FJ-CARLISLE 708               3102
FJ-FRANKLIN 661               2770
FJ-MYERSTOWN 518              2370
PILOT JASPER 500              2231
PILOT TOMS BROOK 4642         1830
PILOT FLYING J SOUTHEAST      1820
PILOT HEBRON 285              1780
PILOT #4599                   1732
PILOT GREENEVILLE 051         1724
PILOT CROSSVILLE 114          1683
PILOT RAPHINE 4649            1667
PILOT MURFREESBORO 404        1538
FJ-CLEAR BROOK 752            1458
FJ-WYTHEVILLE 754             1418
PILOT LEBANON 411             1273
FJ-JEFFERSONVILLE 698         1270
PILOT LEBANON JUNCTION 39     1254
FJ-MILLERSPORT 699            1252
PILOT WYTHEVILLE 4619         1235
FJ-KNOXVILLE 722              1148
PILOT HARRISONBURG 491        1142
FJ-MAX MEADOWS 750            1137
PILOT 219                     1134
PILOT #4596                   1118
PILOT 179                     1102
CAT SCALES PILOT TRAVEL C     1062
PILOT GLENDALE 048  

In [67]:
data_fuel_transaction.shape

(147861, 20)

In [68]:
data_fuel_events.head(n=5)

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2


In [69]:
data_fuel_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8495130 entries, 0 to 8495129
Data columns (total 15 columns):
Id                   int64
ESS_Id               int64
EventTimeStamp       object
EquipmentID          int64
MCTNumber            int64
EquipmentType        object
DriverID             object
Latitude             float64
Longitude            float64
LocationTimeStamp    object
Speed                int64
Heading              float64
Odometer             float64
IgnitionStatus       int64
EFReportReason       int64
dtypes: float64(4), int64(7), object(4)
memory usage: 972.2+ MB


In [70]:
data_fuel_events.isnull().any()

Id                   False
ESS_Id               False
EventTimeStamp       False
EquipmentID          False
MCTNumber            False
EquipmentType        False
DriverID              True
Latitude             False
Longitude            False
LocationTimeStamp    False
Speed                False
Heading              False
Odometer             False
IgnitionStatus       False
EFReportReason       False
dtype: bool

In [71]:
type(data_fuel_events)

pandas.core.frame.DataFrame

In [72]:
data_fuel_events['DriverID'].value_counts()

DYET          33901
SIMJIM        27204
VAUJAM        25743
ULAL          25584
ECKS          24125
CHAJO         23895
OBRG          23551
BULR          23042
CUNW          22963
RENT          22957
LOVH          22395
BROD03        22313
PERT          22190
FARMI         22039
STEGA         21968
CHRN          21921
NARJ          21574
GALJU         21530
SWEJ          21140
ANTR          21046
TAYJON        20882
EDWTI         20845
SAVE          20688
FERCA         20550
SIMJE         20529
STOD          20443
DINE          20246
MORJAM        20216
CRES          19957
JENLE         19871
              ...  
SHOP3            52
BANJA            51
CARJ01           48
DENJE            48
REERO            42
STAW             41
SNID             22
ADADE            20
RUDD             17
MEID             12
JOHLO            11
SCHJ             10
CRAIG B          10
HARGR             2
BYRCHI            2
KINCA             2
GODJO             2
SHOP!             2
WILRON            2


In [73]:
data_fuel_events['Odometer'].max()

861003.90000000002

In [74]:
data_fuel_events.loc[data_fuel_events['Odometer'] == 861003.90000000002]


,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
7829068,20484418,40248138,2018-02-17 11:31:56.697,1664,105438725,tractor,HERD,33.02991,-84.96272,2018-02-17 11:31:57.000,0,289.7,861003.9,1,1


In [100]:
len(data_fuel_events['DriverID'].unique())

949

In [76]:
data_fuel_events['Speed'].value_counts()

0      5297851
67      329702
2       169310
65      159735
66      146349
3       134205
4       128776
1       110966
5       104669
6        92148
64       85300
7        80793
8        75987
63       66388
9        62457
62       60094
68       59968
61       55696
60       55437
10       54146
59       50708
58       48090
11       47974
57       44326
56       41349
12       38793
55       37598
54       32557
13       30631
52       28734
        ...   
38       15817
27       15609
26       15309
32       15074
34       14991
24       14911
23       14900
33       14864
36       14822
37       14809
25       14773
35       14596
70       12334
71        7497
72        4488
73        2680
74        1494
75         935
76         486
158        390
77         263
78         151
79          61
80          42
81          21
83          12
82          11
84           3
88           1
143          1
Name: Speed, Length: 88, dtype: int64

In [77]:
data_fuel_tank_levels.head(n=5)

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [78]:
data_fuel_tank_levels[data_fuel_tank_levels.TankId == 2]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
48670,10633484,2,5.5,11.00,12933194
1824799,14078949,2,1.8,3.60,16673494
4579414,13743578,2,5.5,11.00,16313459
5656380,13899589,2,1.8,4.14,16481275


In [79]:
data_fuel_tank_levels.isnull().any()

Id                  False
TankId              False
TankLevelPercent    False
TankLevelGallons    False
ExactFuelEventId    False
dtype: bool

In [80]:
data_fuel_events.isnull().sum()

Id                        0
ESS_Id                    0
EventTimeStamp            0
EquipmentID               0
MCTNumber                 0
EquipmentType             0
DriverID             318942
Latitude                  0
Longitude                 0
LocationTimeStamp         0
Speed                     0
Heading                   0
Odometer                  0
IgnitionStatus            0
EFReportReason            0
dtype: int64

In [81]:
print(data_units.head(n=15))

    unit_number model_year          make
0         535.0       1996     PETERBILT
1         534.0       1996     PETERBILT
2         533.0       1996     PETERBILT
3         532.0       1996     PETERBILT
4         531.0       1996     PETERBILT
5         526.0       1996     PETERBILT
6         238.0       1995     PETERBILT
7         536.0       1997     PETERBILT
8         537.0       1997     PETERBILT
9         538.0       1997     PETERBILT
10        539.0       1997     PETERBILT
11        540.0       1997     PETERBILT
14        121.0       1999     PETERBILT
15       1505.0       2012  FREIGHTLINER
16       1506.0       2012  FREIGHTLINER


In [83]:
data_fuel_transaction.iloc[5:20,5:]

,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
5,BATK,1992,ULSD,93.88,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 03:07:00
6,BATK,1992,SCLE,1.00,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 03:15:00
7,MARJOH,1730,ULSD,136.29,DEFD,5.50,,0.0,,0.0,,0.0,,0,2017-01-01 04:30:00
8,JAML,1672,ULSD,149.04,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 04:30:00
9,JAML,1672,CADV,1.00,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 04:37:00
10,BEAKEV,1739,ULSD,99.85,CADV,1.00,,0.0,,0.0,,0.0,,0,2017-01-01 04:46:00
11,MCAD,1585,ULSD,131.69,CADV,1.00,,0.0,,0.0,,0.0,,0,2017-01-01 04:53:00
12,JARJ,1930,ULSD,90.62,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 05:16:00
13,LESJ,1608,ULSD,89.72,DEFD,3.58,CADV,1.0,,0.0,,0.0,,0,2017-01-01 05:20:00
14,BLACH,1878,ULSD,61.52,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 05:30:00


In [ ]:
data_fuel_transaction.groupby(['TRNST','TRNSTC','TRNTSN']).size()

In [ ]:
data_fuel_events.head(n=5)

In [ ]:
data_fuel_events.groupby(['DriverID','Speed']).size().to_frame('size').reset_index().sort_values(['DriverID', 'size'], ascending=[True, False])
#data_fuel_transaction.groupby(['TRNST','TRNSTC','TRNTSN']).count()

In [87]:
data_fuel_tank_levels.head(n=5)

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [88]:
data_fuel_tank_levels['TankId'].value_counts()

1    7908824
2          4
Name: TankId, dtype: int64

In [89]:
data_fuel_tank_levels[data_fuel_tank_levels['TankId'] == 2]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
48670,10633484,2,5.5,11.00,12933194
1824799,14078949,2,1.8,3.60,16673494
4579414,13743578,2,5.5,11.00,16313459
5656380,13899589,2,1.8,4.14,16481275


In [ ]:
# Datetime format reference: 
# https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior 
##########---------converting cols to datetime-------

def cols_to_timestamp(df, col_list):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format = '%Y-%m-%d %H:%M:%S')
    return df
#ts_cols = ['EventTimeStamp', 'LocationTimeStamp']
#events_fuel_df =  cols_to_timestamp(events_fuel_df, ts_cols) 

In [92]:
#data performance 
data_performance = pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv')


In [95]:
data_performance.shape

(32387, 32)

In [97]:
data_performance.columns

Index(['Id', 'ESS_Id', 'EventTimeStamp', 'EquipmentID', 'MCTNumber',
       'EquipmentType', 'DriverID', 'Driver2ID', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'IgnitionStatus', 'TripStatus',
       'DataStartTimeStamp', 'DataEndTimeStamp', 'Distance', 'DriveTime',
       'EngineTime', 'MoveTime', 'IntertripIdleTime', 'ShortIdleTime',
       'OverSpeedTime', 'ExcessSpeedTime', 'TotalFuelUsed', 'IdleFuelUsed',
       'FaultFlag', 'RegisteredDriver', 'CruiseControlTime', 'TopGearTime',
       'GearDataSource', 'SpeedUnits', 'SpeedMatrix'],
      dtype='object')

In [99]:
data_performance.iloc[ : , 10 :20]

,LocationTimeStamp,IgnitionStatus,TripStatus,DataStartTimeStamp,DataEndTimeStamp,Distance,DriveTime,EngineTime,MoveTime,IntertripIdleTime
0,2017-01-01 07:29:45.000,1,O,2016-12-23 16:09:00.000,2017-01-01 07:29:00.000,0,3,26,3,22
1,2017-01-01 07:29:45.000,1,O,2016-12-20 04:06:00.000,2016-12-23 16:09:00.000,1617,1950,3678,1808,1716
2,2017-01-01 07:49:39.000,2,O,2016-11-08 04:23:00.000,2017-01-01 07:37:00.000,1720,2072,2193,1988,88
3,2017-01-01 13:58:13.000,2,O,2016-12-22 18:35:00.000,2017-01-01 13:57:00.000,0,7,7,5,0
4,2017-01-01 13:58:13.000,2,O,2016-12-20 04:06:00.000,2016-12-22 18:35:00.000,1474,1761,1793,1690,8
5,2017-01-01 14:19:50.000,2,O,2016-12-20 04:06:00.000,2017-01-01 14:19:00.000,2059,2308,2367,2190,0
6,2017-01-01 16:00:24.000,2,O,2016-12-23 16:30:00.000,2017-01-01 15:59:00.000,0,6,27,4,17
7,2017-01-01 16:00:24.000,2,O,2016-12-20 04:06:00.000,2016-12-23 16:30:00.000,2221,2447,2498,2365,0
8,2017-01-01 18:09:35.000,1,O,2016-12-13 04:39:00.000,2017-01-01 18:09:00.000,0,0,2,0,0
9,2017-01-01 18:18:14.000,1,O,2016-12-22 15:21:00.000,2017-01-01 18:11:00.000,1,13,21,6,0


In [105]:
data_fuel_transaction.columns


Index(['tr_id', 'tr_station_id', 'tr_station_name', 'tr_station_cty',
       'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 'tr_item_cd1',
       'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2', 'tr_item_cd3',
       'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4', 'tr_item_cd5',
       'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6', 'tr_datetime'],
      dtype='object')

In [141]:
df_ulal_events = data_fuel_events[data_fuel_events['DriverID'] == 'ULAL']


In [145]:
df_ulal_events

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2
44,12649143,13640332,2017-01-01 00:12:23.187,1564,105351785,tractor,ULAL,39.90267,-84.19318,2017-01-01 00:12:21.000,9,167.9,498691.1,1,1
45,12649144,13640333,2017-01-01 00:14:13.187,1564,105351785,tractor,ULAL,39.90259,-84.19231,2017-01-01 00:14:11.000,0,0.0,498691.2,2,1
46,12649145,13640334,2017-01-01 00:14:34.267,1564,105351785,tractor,ULAL,39.90262,-84.19234,2017-01-01 00:14:32.000,0,0.0,498691.2,2,0
47,12649146,13640335,2017-01-01 00:32:13.267,1564,105351785,tractor,ULAL,39.90261,-84.19228,2017-01-01 00:32:11.000,0,0.0,498691.2,2,1
48,12649147,13640336,2017-01-01 00:36:23.297,1564,105351785,tractor,ULAL,39.90302,-84.19245,2017-01-01 00:36:21.000,9,271.1,498691.2,1,0
49,12649148,13640337,2017-01-01 00:51:23.390,1564,105351785,tractor,ULAL,39.75214,-84.18651,2017-01-01 00:51:21.000,57,77.4,498703.8,1,2
138,12649244,13640439,2017-01-01 01:51:23.623,1564,105351785,tractor,ULAL,39.42987,-83.17320,2017-01-01 01:51:22.000,65,113.6,498766.5,1,2
199,12649305,13640488,2017-01-01 02:22:33.763,1564,105351785,tractor,ULAL,39.16550,-82.73743,2017-01-01 02:22:33.000,0,0.0,498798.4,1,2
200,12649306,13640489,2017-01-01 02:24:13.763,1564,105351785,tractor,ULAL,39.16544,-82.73734,2017-01-01 02:24:13.000,0,0.0,498798.4,2,1


In [142]:
df_ulal_tr = data_fuel_transaction[data_fuel_transaction['tr_drvr_code'] == 'ULAL  ']

In [189]:
df_ulal_tr.columns

Index(['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
       'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id',
       'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
       'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
       'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6',
       'tr_datetime'],
      dtype='object')

In [194]:
columns_del_tr = ['tr_date','tr_time','tr_unit_id','tr_item_qty2','tr_item_qt3','tr_item_qty4','tr_item_qty5','tr_item_qty6','tr_item_cd3','tr_item_cd4','tr_item_cd5','tr_item_cd6']

In [201]:
def col_del_fun(df,col):
    df = df.drop(col,axis=1)
    return df

In [204]:
df_ulal_t = col_del_fun(df_ulal_tr,'tr_time')

In [205]:
df_ulal_t

,tr_id,tr_date,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,...,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,20170101,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,1.00,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 00:20:00
110,42614106,20170101,516130,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,76.87,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 17:12:00
282,42672413,20170102,524100,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,12.37,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-02 15:29:00
851,42873010,20170104,516012,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,50.58,...,1.00,,0.0,,0.0,,0.0,,0,2017-01-04 11:52:00
883,42882523,20170104,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,141.73,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-04 13:00:00
2089,43280882,20170107,231064,FJ-BEAVER DAM 695,BEAVER DAM,OH,ULAL,1564,ULSD,138.12,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-07 15:20:00
2417,43394310,20170109,231060,FJ-CARLISLE 708,CARLISLE,PA,ULAL,1564,SCLE,1.00,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-09 02:03:00
3041,43596583,20170110,231846,FJ-MAX MEADOWS 750,FORT CHISWELL,VA,ULAL,1564,ULSD,171.05,...,6.92,CADV,1.0,,0.0,,0.0,,0,2017-01-10 15:09:00
3424,43719948,20170111,521442,TIMEOUT MARKET #22 & TC,NEWPORT,TN,ULAL,1564,SCLE,1.00,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-11 13:39:00
4302,44020016,20170113,516264,PILOT MURFREESBORO 404,MURFREESBORO,TN,ULAL,1564,ULSD,100.24,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-13 16:45:00


In [ ]:
#df_ulal_performance = df_ulal_performance.drop('SpeedMatrix', axis=1)

In [207]:
df_ulal_tr

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,20170101,0020,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 00:20:00
110,42614106,20170101,1712,516130,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 17:12:00
282,42672413,20170102,1529,524100,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-02 15:29:00
851,42873010,20170104,1152,516012,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,...,1.00,,0.0,,0.0,,0.0,,0,2017-01-04 11:52:00
883,42882523,20170104,1300,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-04 13:00:00
2089,43280882,20170107,1520,231064,FJ-BEAVER DAM 695,BEAVER DAM,OH,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-07 15:20:00
2417,43394310,20170109,0203,231060,FJ-CARLISLE 708,CARLISLE,PA,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-09 02:03:00
3041,43596583,20170110,1509,231846,FJ-MAX MEADOWS 750,FORT CHISWELL,VA,ULAL,1564,ULSD,...,6.92,CADV,1.0,,0.0,,0.0,,0,2017-01-10 15:09:00
3424,43719948,20170111,1339,521442,TIMEOUT MARKET #22 & TC,NEWPORT,TN,ULAL,1564,SCLE,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-11 13:39:00
4302,44020016,20170113,1645,516264,PILOT MURFREESBORO 404,MURFREESBORO,TN,ULAL,1564,ULSD,...,0.00,,0.0,,0.0,,0.0,,0,2017-01-13 16:45:00


In [182]:
df_ulal_performance = df_ulal_performance.drop('SpeedMatrix', axis=1)

In [183]:
df_ulal_performance.dtypes

Id                      int64
ESS_Id                  int64
EventTimeStamp         object
DriverID               object
Latitude              float64
Longitude             float64
LocationTimeStamp      object
DataStartTimeStamp     object
DataEndTimeStamp       object
Distance                int64
DriveTime               int64
EngineTime              int64
MoveTime                int64
IntertripIdleTime       int64
ShortIdleTime           int64
OverSpeedTime           int64
TotalFuelUsed         float64
IdleFuelUsed          float64
CruiseControlTime       int64
TopGearTime             int64
dtype: object

In [184]:
df_ulal_performance = data_performance[data_performance['DriverID']=='ULAL']

In [185]:
df_ulal_performance
#13640272

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Driver2ID,Latitude,Longitude,...,ExcessSpeedTime,TotalFuelUsed,IdleFuelUsed,FaultFlag,RegisteredDriver,CruiseControlTime,TopGearTime,GearDataSource,SpeedUnits,SpeedMatrix
98,76916,13659755,2017-01-03 04:24:35.000,1564,105351785,tractor,ULAL,NaN,35.050694,-80.618379,...,0,378.5,7.5,0,1,1607,2119,3,MPH,"94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
751,77569,13803666,2017-01-10 04:27:49.000,1564,105351785,tractor,ULAL,NaN,37.876018,-79.308101,...,0,378.5,90.2,0,1,1119,1577,3,MPH,"41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
1390,78208,13942284,2017-01-17 04:19:11.000,1564,105351785,tractor,ULAL,NaN,35.755555,-80.347824,...,0,396.1,33.2,0,1,1526,2010,3,MPH,"67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
2004,78822,14082968,2017-01-24 04:18:37.000,1564,105351785,tractor,ULAL,NaN,39.165138,-82.736574,...,0,371.5,72.7,1,1,1065,1613,3,MPH,"79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
2621,79439,14218850,2017-01-31 04:13:45.000,1564,105351785,tractor,ULAL,NaN,39.951759,-83.378240,...,0,318.0,30.7,0,1,990,1528,3,MPH,"115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3501,80319,14357577,2017-02-07 04:15:51.000,1564,105351785,tractor,ULAL,NaN,37.414814,-85.881620,...,0,381.0,12.2,0,1,1369,1893,3,MPH,"49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
3811,80629,14499174,2017-02-14 04:15:32.000,1564,105351785,tractor,ULAL,NaN,40.123240,-74.709490,...,0,412.7,73.6,0,1,875,1583,3,MPH,"53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
4857,81675,14643718,2017-02-21 05:00:39.000,1564,105351785,tractor,ULAL,NaN,41.659259,-93.500648,...,0,430.3,62.0,0,1,1170,1888,3,MPH,"41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
5072,81890,14784110,2017-02-28 04:20:35.000,1564,105351785,tractor,ULAL,NaN,33.580324,-83.576296,...,0,428.7,27.6,0,1,1613,2248,3,MPH,"29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."
5629,82447,14923136,2017-03-07 04:17:25.000,1564,105351785,tractor,ULAL,NaN,40.925092,-74.791944,...,0,392.6,77.2,0,1,941,1583,3,MPH,"68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."


In [152]:
data_fuel_tank_levels[data_fuel_tank_levels['ExactFuelEventId'] == 12649103]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
199091,10375469,1,50.3,125.75,12649103


In [153]:
data_fuel_tank_levels[data_fuel_tank_levels['ExactFuelEventId'] == 12649143]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
205276,10375509,1,50.6,126.5,12649143
